In [9]:
import os
import numpy as np
from PIL import Image

def parse_annotation(labels_path):
    objects = []
    with open(labels_path, 'r') as file:
        for line in file.readlines():
            parts = line.strip().split()
            class_id = int(parts[0])
            coordinates = list(map(float, parts[1:]))  # Convert all to float
            xs = coordinates[0::2]  # Extract all x coordinates
            ys = coordinates[1::2]  # Extract all y coordinates
            xmin, xmax = min(xs), max(xs)
            ymin, ymax = min(ys), max(ys)
            objects.append((class_id, xmin, ymin, xmax, ymax))
    return objects

# def check_annotation_example(annotation_dir):
#     example_annotation_file = next(os.path.join(annotation_dir, f) for f in os.listdir(annotation_dir) if f.endswith('.txt'))
#     with open(example_annotation_file, 'r') as file:
#         for i, line in enumerate(file):
#             print(f"Line {i + 1}: {line.strip()}")
#             if i >= 5:  # Print only the first 5 lines
#                 break
# 
# # Call this function to check the format
# annotation_dir = os.path.join(r'C:/Users/SirM/Desktop/Swayam/Intro to Deep Learning/Intro-to-Deep-Learning/Final Project/PKLot.v1-raw.yolov8-obb', 'train', 'labels')  # Adjust as necessary
# check_annotation_example(annotation_dir)

def resize_image(image, new_width, new_height, boxes):
    width, height = image.size
    resize_ratio_w = new_width / width
    resize_ratio_h = new_height / height
    image = image.resize((new_width, new_height))
    
    resized_boxes = []
    for box in boxes:
        class_id, x1, y1, x2, y2 = box
        resized_boxes.append((
            class_id,
            x1 * resize_ratio_w,
            y1 * resize_ratio_h,
            x2 * resize_ratio_w,
            y2 * resize_ratio_h
        ))
    return image, resized_boxes

def preprocess_image(image_path, annotation_path, target_size=(128, 128)):
    image = Image.open(image_path)
    boxes = parse_annotation(annotation_path)
    
    image, boxes = resize_image(image, target_size[0], target_size[1], boxes)
    image_np = np.array(image) / 255.0  # Normalize pixel values
    
    return image_np, boxes

def process_directory(data_dir, annotation_dir, target_size=(128, 128)):
    processed_data = []
    for img_filename in os.listdir(data_dir):
        if img_filename.endswith('.jpg'):
            image_path = os.path.join(data_dir, img_filename)
            annotation_path = os.path.join(annotation_dir, img_filename.replace('.jpg', '.txt'))
            processed_image, processed_boxes = preprocess_image(image_path, annotation_path, target_size)
            processed_data.append((processed_image, processed_boxes))
    return processed_data

# Example usage
dataset_base = r'C:/Users/SirM/Desktop/Swayam/Intro to Deep Learning/Intro-to-Deep-Learning/Final Project/PKLot.v1-raw.yolov8-obb'
partitions = ['train', 'valid', 'test']
target_size = (128, 128)  # Change as required by your model

all_data = {}
for part in partitions:
    print(f"Processing {part} data...")
    images_dir = os.path.join(dataset_base, part, 'images')
    annotations_dir = os.path.join(dataset_base, part, 'labels')  # Adjust if different
    all_data[part] = process_directory(images_dir, annotations_dir, target_size)
    print(f"Finished processing {len(all_data[part])} images from {part} set.")


Line 1: 1 0.2171875 0.2569444444444444 0.253125 0.2569444444444444 0.253125 0.3194444444444444 0.2171875 0.3194444444444444
Line 2: 0 0.2421875 0.2569444444444444 0.27734375 0.2569444444444444 0.27734375 0.3236111111111111 0.2421875 0.3236111111111111
Line 3: 1 0.26953125 0.2569444444444444 0.303125 0.2569444444444444 0.303125 0.3236111111111111 0.26953125 0.3236111111111111
Line 4: 0 0.29296875 0.25555555555555554 0.32890625 0.25555555555555554 0.32890625 0.32222222222222224 0.29296875 0.32222222222222224
Line 5: 1 0.31953125 0.25972222222222224 0.353125 0.25972222222222224 0.353125 0.32222222222222224 0.31953125 0.32222222222222224
Line 6: 1 0.36015625 0.25555555555555554 0.39296875 0.25555555555555554 0.39296875 0.3194444444444444 0.36015625 0.3194444444444444
Processing train data...
Finished processing 8691 images from train set.
Processing valid data...
Finished processing 2483 images from valid set.
Processing test data...
Finished processing 1242 images from test set.
